#Import

In [ ]:
! pip install datasets transformers[sentencepiece]
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 14.4 MB/s 
     |████████████████████████████████| 5.3 MB 58.9 MB/s 
     |████████████████████████████████| 163 kB 72.4 MB/s 
     |████████████████████████████████| 115 kB 70.0 MB/s 
     |████████████████████████████████| 212 kB 72.4 MB/s 
     |████████████████████████████████| 127 kB 56.4 MB/s 
     |████████████████████████████████| 7.6 MB 55.9 MB/s 
     |████████████████████████████████| 1.3 MB 53.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset, load_metric
from transformers import *
import numpy as np
import torch
import random

##parameter setting

In [ ]:
# Set up all parameter
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# data parameter
chunk_size = 128 # size of down sample data
SIZE = 6 # size of k-mer

# model parameter
vocab_size = 4**SIZE + 5 # 4101 # number of vocab.txt in tokenizer (4^SIZE + 5)
max_length = 512 # universal standard
pretrain_model = "armheb/DNA_bert_" + str(SIZE)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## load data

In [ ]:
# don't forget set seed
# don't forget change dataset to omicron
PATH = "/content/drive/MyDrive/New Covid19_Data/4.1_implement_model/4_1 101 seq For Train.csv"
data_files = {"train": PATH,"validation": PATH }
Datasets = load_dataset("csv", data_files=data_files , index_col=0)
Datasets = Datasets.rename_column("__index_level_0__", "label")
# Datasets = Datasets['train'].train_test_split(test_size=0.2)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
lit = []
for i in Datasets['train']['Sequence']:
  lit.append(len(i))
print ("max lenght sequence is", max(lit))
print ("min lenght sequence is", min(lit))

max lenght sequence is 29946
min lenght sequence is 29946


##Set tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrain_model)
# tokenizer = BertTokenizer.from_pretrained('armheb/DNA_bert_6')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--armheb--DNA_bert_6/snapshots/a79a8fd96ad172f964a4dbef3f4d7545a5034baa/config.json
Model config BertConfig {
  "_name_or_path": "armheb/DNA_bert_6",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_rnn_layer": 1,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rnn": "lstm",
  "rnn_dropout": 0.0,
  "rnn_hidden": 768,
  "split": 10,
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4101
}



Downloading:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--armheb--DNA_bert_6/snapshots/a79a8fd96ad172f964a4dbef3f4d7545a5034baa/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--armheb--DNA_bert_6/snapshots/a79a8fd96ad172f964a4dbef3f4d7545a5034baa/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--armheb--DNA_bert_6/snapshots/a79a8fd96ad172f964a4dbef3f4d7545a5034baa/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--armheb--DNA_bert_6/snapshots/a79a8fd96ad172f964a4dbef3f4d7545a5034baa/config.json
Model config BertConfig {
  "_name_or_path": "armheb/DNA_bert_6",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_

## create function

In [ ]:
cal = SIZE - 1  - 2 # formula is n+5 (kmer 6 mean dna increase from original 5 dna)
start_pos = 70   # start from 0 position

def getKmers(sequence, size=SIZE): # return to string
  seq = ""
  for x in range(len(sequence) - size + 1):
    seq += sequence[x:x+size] + " "
  return seq

def split_seq(series):
    seq_split = []
    for i in series:
        # if len(i) >= 28672:
        seq_split.append(getKmers(i[start_pos:29000 - cal + start_pos]))  # should be 28672 when cut all sequence
            # it will increase 2 because k-mer it drive to more token in sequence
    return seq_split

In [ ]:
# test split_seq function
len(split_seq(Datasets['train']['Sequence'])), len(split_seq(Datasets['train']['Sequence'][:5])[0].split())

(100, 28992)

(100, 28992) True

In [ ]:
def tokenize_function(examples):
    result = tokenizer(split_seq(examples["Sequence"]) ,truncation=True,
                    padding='max_length', max_length=29000)
    # if tokenizer.is_fast:
    #     result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# manipulate data

In [ ]:
# Use batched=True to activate fast multithreading!
tokenized_datasets = Datasets.map(
    tokenize_function, batched=True, remove_columns=['Sequence', 'label']
)
tokenized_datasets

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [ ]:
len(tokenized_datasets['train']['input_ids'][0])

29000

In [ ]:
tokenized_datasets['train']['input_ids'][0][:200]

[2,
 711,
 2830,
 3114,
 154,
 601,
 2389,
 1349,
 1285,
 1030,
 11,
 30,
 108,
 418,
 1660,
 2530,
 1916,
 3556,
 1923,
 3582,
 2028,
 4002,
 3707,
 2525,
 1895,
 3470,
 1579,
 2208,
 628,
 2499,
 1790,
 3052,
 4003,
 3709,
 2534,
 1932,
 3619,
 2174,
 490,
 1945,
 3672,
 2386,
 1340,
 1251,
 893,
 3559,
 1934,
 3627,
 2205,
 615,
 2448,
 1587,
 2237,
 744,
 2962,
 3641,
 2262,
 841,
 3349,
 1094,
 266,
 1049,
 85,
 326,
 1289,
 1045,
 71,
 270,
 1065,
 149,
 582,
 2314,
 1049,
 87,
 334,
 1324,
 1186,
 635,
 2528,
 1906,
 3514,
 1756,
 2913,
 3447,
 1485,
 1832,
 3220,
 577,
 2295,
 973,
 3879,
 3216,
 561,
 2232,
 722,
 2873,
 3285,
 839,
 3342,
 1067,
 160,
 626,
 2491,
 1758,
 2921,
 3478,
 1611,
 2334,
 1130,
 411,
 1630,
 2412,
 1443,
 1661,
 2536,
 1940,
 3651,
 2302,
 1004,
 4003,
 3710,
 2538,
 1945,
 3671,
 2384,
 1332,
 1218,
 762,
 3034,
 3931,
 3424,
 1394,
 1467,
 1759,
 2928,
 3506,
 1724,
 2786,
 2938,
 3548,
 1891,
 3453,
 1512,
 1939,
 3647,
 2288,
 945,
 3766,
 2763

In [ ]:
# chunk_size = 128 # try to set small chunk size to 128

def group_texts(examples):
    # Group all text
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # measure all text
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # we will chose chunk 1 till before end (ถ้า chunk สุดท้ายเล็กกว่าที่กำหนดไว้)
    total_length = (total_length // chunk_size) * chunk_size
    # แบ่ง Chunk
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # create Labels
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 22656
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 22656
    })
})

In [ ]:
# split and downsample data

all_size = lm_datasets['train'].num_rows # train full data
# all_size = 10_000
train_size = int(0.8 * all_size)
test_size = int(0.2 * all_size) # why l select 0.1 before

downsampled_dataset = lm_datasets["train"].train_test_split(
  train_size=train_size, test_size=test_size, shuffle=False, seed=RANDOM_SEED
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 18124
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4531
    })
})

In [ ]:
# check validation
downsampled_dataset['test'].num_rows /(downsampled_dataset['train'].num_rows + downsampled_dataset['test'].num_rows)

0.2

downsampling technique because it have such a problem cuda out of runtime

#Set model

In [ ]:
# masking 15 percent by collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
# set bert base model config
# bert_base_config = BertConfig(
#     vocab_size=vocab_size,
#     max_position_embeddings=max_length,
#     )
# model = BertForMaskedLM(config=bert_base_config)

# set config bigbird model
# model_bigbert_config = BigBirdConfig(
#     vocab_size=vocab_size,
#     max_position_embeddings=max_length,
# )
# model = BigBirdForMaskedLM(config=model_bigbert_config)

my bert less than original bert because vacab size and max lenght not equal

In [ ]:
# set config pretrain model
model = AutoModelForMaskedLM.from_pretrained(pretrain_model)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--armheb--DNA_bert_6/snapshots/a79a8fd96ad172f964a4dbef3f4d7545a5034baa/config.json
Model config BertConfig {
  "_name_or_path": "armheb/DNA_bert_6",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_rnn_layer": 1,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rnn": "lstm",
  "rnn_dropout": 0.0,
  "rnn_hidden": 768,
  "split": 10,
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4101
}

loading weights file pytorch_model.bin f

In [ ]:
model_parameter = model.num_parameters() / 1_000_000
print(f"'>>> model parameter: {round(model_parameter)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> model parameter: 89M'
'>>> BERT number of parameters: 110M'


#Train model

In [ ]:
# push to hub choice
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = 'BERT'

# check training argument # check push to hub
training_args = TrainingArguments(
    output_dir=f"{model_name}-{SIZE}_tokenized",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # push_to_hub=True,
    fp16=True, # สำหรับคนใช้ GPU
    logging_steps=logging_steps,
    num_train_epochs=10,
    seed=RANDOM_SEED
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 18124
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2840
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.075800,0.040871
2,0.043900,0.039330
3,0.040100,0.039908
4,0.039700,0.037474
5,0.038500,0.037917
6,0.037300,0.040119
7,0.037300,0.038378
8,0.038300,0.037932
9,0.036900,0.036502
10,0.037500,0.036392


***** Running Evaluation *****
  Num examples = 4531
  Batch size = 64
Saving model checkpoint to BERT-6_tokenized/checkpoint-500
Configuration saved in BERT-6_tokenized/checkpoint-500/config.json
Model weights saved in BERT-6_tokenized/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4531
  Batch size = 64
***** Running Evaluation *****
  Num examples = 4531
  Batch size = 64
Saving model checkpoint to BERT-6_tokenized/checkpoint-1000
Configuration saved in BERT-6_tokenized/checkpoint-1000/config.json
Model weights saved in BERT-6_tokenized/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4531
  Batch size = 64
***** Running Evaluation *****
  Num examples = 4531
  Batch size = 64
Saving model checkpoint to BERT-6_tokenized/checkpoint-1500
Configuration saved in BERT-6_tokenized/checkpoint-1500/config.json
Model weights saved in BERT-6_tokenized/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examp

TrainOutput(global_step=2840, training_loss=0.04251042300126922, metrics={'train_runtime': 1461.2337, 'train_samples_per_second': 124.032, 'train_steps_per_second': 1.944, 'total_flos': 1.192213246768128e+16, 'train_loss': 0.04251042300126922, 'epoch': 10.0})

i expect loss so vary because i shuffle validation data on train test split function

In [ ]:
import math
import numpy as np
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.4f}")

***** Running Evaluation *****
  Num examples = 4531
  Batch size = 64


>>> Perplexity: 1.0401


## push to hub

In [ ]:
# trainer.push_to_hub()

Saving model checkpoint to BERT_full-6_tokenized
Configuration saved in BERT_full-6_tokenized/config.json
Model weights saved in BERT_full-6_tokenized/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/340M [00:00<?, ?B/s]

Upload file runs/Oct21_07-29-49_60584ca57018/events.out.tfevents.1666341045.60584ca57018.74.2: 100%|##########…

Upload file runs/Oct21_07-29-49_60584ca57018/events.out.tfevents.1666337415.60584ca57018.74.0:  23%|##2       …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Mozart-coder/BERT_full-6_tokenized
   cf96f79..25806ee  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Mozart-coder/BERT_full-6_tokenized
   cf96f79..25806ee  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
To https://huggingface.co/Mozart-coder/BERT_full-6_tokenized
   25806ee..5bfdb1f  main -> main

   25806ee..5bfdb1f  main -> main



'https://huggingface.co/Mozart-coder/BERT_full-6_tokenized/commit/25806ee383e7bfa2e3416903a669dfe416df5de2'

In [ ]:
tokenizer.push_to_hub("Mozart-coder/BERT_full-6_tokenized")

tokenizer config file saved in /tmp/tmp2bryy9xx/tokenizer_config.json
Special tokens file saved in /tmp/tmp2bryy9xx/special_tokens_map.json
Uploading the following files to Mozart-coder/BERT_full-6_tokenized: tokenizer.json,vocab.txt,tokenizer_config.json,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/Mozart-coder/BERT_full-6_tokenized/commit/f21bd4d6e0c3c7ca862fe018530f86b5f7e05df8', commit_message='Upload tokenizer', commit_description='', oid='f21bd4d6e0c3c7ca862fe018530f86b5f7e05df8', pr_url=None, pr_revision=None, pr_num=None)

# Evaluation

In [ ]:
! pip install nltk rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 18124
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4531
    })
})

In [ ]:
! git lfs install.


Error: unknown command "install." for "git-lfs"

Did you mean this?
	install

Run 'git-lfs --help' for usage.


In [ ]:
predictions = trainer.predict(downsampled_dataset["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 4531
  Batch size = 64


(4531, 128, 4101) (4531, 128)


In [ ]:
metric = load_metric("glue", "mrpc")
rouge = load_metric("rouge")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [ ]:
logits = predictions.predictions
labels = predictions.label_ids
pred = np.argmax(logits, axis=-1)

In [ ]:
predf = pred.flatten()
labelsf = labels.flatten()

In [ ]:
re = rouge.compute(predictions=predf, references=labelsf)

KeyboardInterrupt: ignored

In [ ]:
labelsf.shape, predf.shape

((579968,), (579968,))

In [ ]:
labelsf[:100]

array([-100, -100, -100, -100, -100, -100, -100, 2073, -100,  327, -100,
       1063, -100, -100, -100, -100, -100, -100,   83, -100, -100, -100,
       -100, -100,  877, -100, -100, -100, -100,  801, 3191, -100, -100,
       -100, -100, -100, 2058, -100, -100, -100, -100, -100, -100, 3662,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       3305, -100, -100, 2349, 1192, -100, -100, -100, -100, -100, -100,
       -100, -100, -100,  370, -100, -100, -100, -100, -100, -100, -100,
       -100, 3959, 3536, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, 3089,   54, -100, -100, -100,  413, -100,
       -100])

In [ ]:
predf[:100]

array([4026, 3803, 2911, 3437, 1445, 1670, 2570, 2073,   85,  327, 1293,
       1063,  143,  557, 2213,  646, 2569, 2072,   83,  317, 1256,  914,
       3643, 2271,  877, 3496, 1681, 2614, 2252,  801, 3191,  463, 1837,
       3237,  645, 2566, 2058,   28,  100,  387, 1534, 2025, 3991, 3662,
       2345, 1175,  591, 2352, 1201,  693, 2760, 2833, 3128,  211,  830,
       3305,  919, 3663, 2349, 1192,  657, 2615, 2256,  817, 3253,  710,
       2826, 3099,   96,  370, 1468, 1764, 2946, 3580, 2020, 3970, 3580,
       2017, 3959, 3536, 1844, 3266,  761, 3029, 3909, 3333, 1030,   12,
         33,  117,  453, 1800, 3089,   54,  203,  798, 3179,  413, 1640,
       2450])

3803
2911
3437
1445
1670
2570
2073
85
327


In [ ]:
f = open("predsf.txt", "a")
for i in predf:
  f.write(str(i) + "\n")
f.close()

In [ ]:
f = open("labelsf.txt", "a")
for i in labelsf:
  f.write(str(i) + "\n")
f.close()

In [ ]:
predictions.shape

(4531, 128)